# Forte Tutorial 1.02: Forte's determinant class

---

In this tutorial we are going to explore how to create a simple FCI code using forte's Python API.

## Import modules
Here we import `forte.utils` bto access functions to directly run an SCF computation in psi4.

In [1]:
import psi4
import forte
import forte.utils

First we will run psi4 using the function `forte.utils.psi4_scf`

In [2]:
# setup xyz geometry
geom = """
O
H 1 1.0
H 1 1.0 2 180.0
"""
(E_scf, wfn) = forte.utils.psi4_scf(geom,basis='sto-3g',reference='rhf')
print(f'SCF Energy = {E_scf}')

SCF Energy = -74.84159216002942


## Reading options

In [3]:
from forte import forte_options

options = psi4.core.get_options() # options = psi4 option object
options.set_current_module('FORTE') # read options labeled 'FORTE'
forte_options.get_options_from_psi4(options)

## Setting the molecular orbital spaces

In [4]:
# Setup forte and prepare the active space integral class
mos_spaces = {'FROZEN_DOCC' :     [1,0,0,0,0,0,0,0], # freeze the O 1s orbital
              'RESTRICTED_DOCC' : [1,0,0,0,0,1,0,0]}
nmopi = wfn.nmopi()
point_group = wfn.molecule().point_group().symbol()
mo_space_info = forte.make_mo_space_info_from_map(nmopi,point_group,mos_spaces,[])

TypeError: make_mo_space_info_from_map(): incompatible function arguments. The following argument types are supported:
    1. (nmopi: psi4.core.Dimension, point_group: str, mo_space_map: Dict[str, List[int]]) -> forte._forte.MOSpaceInfo

Invoked with: <psi4.core.Dimension object at 0x7fbe85016ff0>, 'd2h', {'FROZEN_DOCC': [1, 0, 0, 0, 0, 0, 0, 0], 'RESTRICTED_DOCC': [1, 0, 0, 0, 0, 1, 0, 0]}, []

In [ ]:
mo_space_info.size('ACTIVE')

## Building a `ForteIntegral` object to read integrals from psi4

In Forte there are two classes responsible for handling integrals:
- `ForteIntegral`: reads the integrals from psi4 and stores them in varios formats (conventional, density fitting, Cholesky, ...).
- `ActiveSpaceIntegrals`: stores a copy of all integrals and it is used by active space methods. This class only stores a subset of the integrals and includes an effective potential due to non-active doubly occupied orbitals.

We will first build the `ForteIntegral` object via the function `make_forte_integrals`

In [ ]:
ints = forte.make_ints_from_psi4(wfn, forte_options, mo_space_info)
print(f'Number of molecular orbitals: {ints.nmo()}')
print(f'Number of correlated molecular orbitals: {ints.ncmo()}')

# the space that defines the active orbitals. We select only the 'ACTIVE' part
active_space = 'ACTIVE'
# the space(s) with non-active doubly occupied orbitals
core_spaces = ['RESTRICTED_DOCC']

as_ints = forte.make_active_space_ints(mo_space_info, ints, active_space, core_spaces)

print(f'Frozen-core energy = {as_ints.frozen_core_energy()}')
print(f'Nuclear repulsion energy = {as_ints.nuclear_repulsion_energy()}')
print(f'Scalar energy = {as_ints.scalar_energy()}')

## Creating determinants

Objects that represent determinants are represented by the class `Determinant`. Here we create an empty determinant and print it by invoking the `str` function. This function prints the entire determinant (which has fixed size), and so if we are working with only a few orbitals we can specify how many we want to print

In [7]:
d = forte.Determinant()
print(f'Determinant: {d}')

# nact = mo_space_info.size('ACTIVE')
nact = 7
print(f'Determinant: {d.str(nact)}')

Determinant: |0000000000000000000000000000000000000000000000000000000000000000>
Determinant: |0000000>


We modify the determinant by applying to it a creation operator $\hat{a}^\dagger_1$ that adds one electron in the spin orbital $\phi_{i,\alpha}$ using the function (`create_alfa_bit`). This function returns the corresponding sign

In [8]:
sign = d.create_alfa_bit(1)
print(f'Determinant: {d.str(nact)}, sign = {sign}')

Determinant: |0+00000>, sign = 1.0


Here we create an electron in orbital 2

In [9]:
sign = d.create_alfa_bit(2)
print(f'Determinant: {d.str(nact)}, sign = {sign}')

Determinant: |0++0000>, sign = -1.0


Similarly, we can remove (annihilate) an electron with the command `destroy_alfa_bit` (`destroy_beta_bit` for the beta case)

In [10]:
sign = d.destroy_alfa_bit(2)
print(f'Determinant: {d.str(nact)}, sign = {sign}')

Determinant: |0+00000>, sign = -1.0


## Creating the HF determinant

Next we do some bookeeping to find out the occupation of the Hartree-Fock determinant using the occupation returned to us by psi4

In [ ]:
nirrep = mo_space_info.nirrep()
nactpi = mo_space_info.dimension('ACTIVE').to_tuple()

# compute the number of alpha electrons per irrep
nact_aelpi = wfn.nalphapi() - mo_space_info.dimension('FROZEN_DOCC') - mo_space_info.dimension('RESTRICTED_DOCC')
nact_aelpi = nact_aelpi.to_tuple()      
# compute the number of beta electrons per irrep
nact_belpi = wfn.nbetapi() - mo_space_info.dimension('FROZEN_DOCC') - mo_space_info.dimension('RESTRICTED_DOCC')
nact_belpi = nact_belpi.to_tuple()           

print(f'Number of alpha electrons per irrep: {nact_aelpi}')
print(f'Number of beta electrons per irrep:  {nact_belpi}')
print(f'Number of active orbtials per irrep: {nactpi}')

ref = forte.Determinant()

# we loop over each irrep and fill the occupied orbitals 
irrep_start = [sum(nactpi[:h]) for h in range(nirrep)]
for h in range(nirrep):
    for i in range(nact_aelpi[h]): ref.set_alfa_bit(irrep_start[h] + i, True)
    for i in range(nact_belpi[h]): ref.set_beta_bit(irrep_start[h] + i, True)        
    
print(f'Reference determinant: {ref.str(nact)}')

We can now compute the energy of the determinant as $\langle \Phi | \hat{H} | \Phi \rangle$ using the `slater_rules` function in the `ActiveSpaceIntegrals` class

In [ ]:
as_ints.slater_rules(ref,ref) + as_ints.scalar_energy() + as_ints.nuclear_repulsion_energy() 

## Creating the FCI determinant basis

Next we enumerate the FCI determinants. Here we use symmetry information and generate only those determinants that have the desired symmetry. We do it in a wasteful way, because we simply generate all the combinations of alpha/beta electrons and then check for the symmetry of the determinant. 

In [ ]:
import itertools
import functools

dets = []
orbs = range(nact)

# get the symmetry of each active orbital
act_sym = mo_space_info.symmetry('ACTIVE')

nact_ael = sum(nact_aelpi)
nact_bel = sum(nact_belpi)
print(f'Number of alpha electrons: {nact_ael}')
print(f'Number of beta electrons:  {nact_bel}')

# specify the target symmetry
sym = 0

# generate all the alpha strings
for astr in itertools.combinations(orbs, nact_ael):
    # compute the symmetry of the alpha string
    asym = functools.reduce(lambda i, j:  i ^ j, [act_sym[x] for x in  astr])
    # generate all the beta strings
    for bstr in itertools.combinations(orbs, nact_bel):
        # compute the symmetry of the beta string
        bsym = functools.reduce(lambda i, j:  i ^ j, [act_sym[x] for x in  bstr])    
        # if the determinant has the correct symmetry save it
        if (asym ^ bsym) == sym:
            d = forte.Determinant()
            for i in astr: d.set_alfa_bit(i, True)
            for i in bstr: d.set_beta_bit(i, True)                
            dets.append(d)

print(f'==> List of FCI determinants <==')
for d in dets:
    print(f'{d.str(4)}')

## Diagonalize the Hamiltonian in the FCI space

In the last step, we diagonalize the Hamiltonian in the FCI determinant basis. We use the function `slater_rules` from the `ActiveSpaceIntegrals` class, which implements Slater rules to compute the matrix elements $\langle \Phi_I | \hat{H} | \Phi_J \rangle$.

In [ ]:
import numpy as np

ndets = len(dets)
H = np.ndarray((ndets,ndets))
for I, detI in enumerate(dets):
    for J, detJ in enumerate(dets):
        H[I][J] = as_ints.slater_rules(detI,detJ)

# or we could use the more fancy looping below that avoid computing half of the matrix elements
# for I, detI in enumerate(dets):
#     H[I][I] = as_ints.slater_rules(detI,detI) # diagonal term
#     for J, detJ in enumerate(dets[:I]):
#         HIJ = as_ints.slater_rules(detI,detJ) # off-diagonal term (only upper half)
#         H[I][J] = H[J][I] = HIJ
   
print(H)
evals, evecs = np.linalg.eigh(H)

psi4_fci = -74.846380133240530
print(f'FCI Energy = {evals[0] + as_ints.scalar_energy() + as_ints.nuclear_repulsion_energy()}')
print(f'FCI Energy Error = {evals[0] + as_ints.scalar_energy() + as_ints.nuclear_repulsion_energy()- psi4_fci}')

index_hf = dets.index(ref)
print(f'Index of the HF determinant in the FCI vector {index_hf}')